In [ ]:
import os
import numpy as np
import pyopencl as cl
import pyopencl.array as cla
from pyopencl.elementwise import ElementwiseKernel as CL_ElK

from pynx.processing_unit.cl_processing_unit import CLProcessingUnit
from pynx.processing_unit.kernel_source import get_kernel_source as getks
pu = CLProcessingUnit()
pu.select_gpu(language='opencl')
pu.init_cl(gpu_name='Iris')

In [ ]:
nb =1000

In [ ]:
a = np.zeros(nb, dtype=np.float32)
cl_a = cla.to_device(pu.cl_queue, a)

k = CL_ElK(pu.cl_ctx, name='test_atomic_add_float', operation="atomic_add_f(a, 1.0f)",
           preamble=getks('opencl/complex.cl'), options= "-cl-mad-enable -cl-fast-relaxed-math",
           arguments="__global float* a")
k(cl_a)
print(cl_a.get()[0])
print(cl_a.get()[0] == nb)

In [ ]:
a = np.zeros(nb, dtype=np.complex64)
cl_a = cla.to_device(pu.cl_queue, a)

k = CL_ElK(pu.cl_ctx, name='test_atomic_add_complex', operation="atomic_add_c(a, (float2)(1.0f, -1.123f))",
           preamble=getks('opencl/complex.cl'), options= "-cl-mad-enable -cl-fast-relaxed-math",
           arguments="__global float2* a")
k(cl_a)
print(cl_a.get()[0])
print(np.isclose(cl_a.get()[0], nb * (1 - 1.123j)))